In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import spikeinterface.full as si
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import savemat
import shutil
import dartsort
import dartsort.vis as dartvis
import torch

In [3]:
darts_dir = Path("~/proj/measims/dart_recordings").expanduser()

In [4]:
dart_out_dir = Path("~/proj/measims/dartnonn4_out").expanduser()
dart_out_dir.mkdir(exist_ok=True)

In [5]:
list(darts_dir.iterdir())

[PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_rigid_bimodal_homogeneous'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_non-rigid_bimodal_modulated'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_non-rigid_uniform_modulated'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_bumps_bimodal_homogeneous'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_bumps_uniform_homogeneous'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_rigid_uniform_homogeneous'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_rigid_uniform_modulated'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_bumps_bimodal_modulated'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_rigid_bimodal_modulated'),
 PosixPath('/home/cwindolf/proj/measims/dart_recordings/Neuropixels-128_bumps_uniform_modulat

In [6]:
cfg = dartsort.DARTsortConfig(
    matching_iterations=0,
    subtract_only=True,
    # hacking the times to use our NN at 32kHz
    waveform_config=dartsort.WaveformConfig(ms_before=1.32, ms_after=2.48),
)

In [7]:
import pickle
def loadpkl(p):
    with open(p, "rb") as f:
        return pickle.load(f)

In [8]:
for p in darts_dir.iterdir():
    rec = si.read_binary_folder(p)
    out_dir = dart_out_dir / p.stem
    out_dir.mkdir(exist_ok=True)
    geom = rec.get_channel_locations()
    channel_index = torch.tensor(dartsort.make_channel_index(geom, 100.0))
    pipeline = dartsort.WaveformPipeline(
        (
            dartsort.transform.TemporalPCADenoiser(channel_index),
            dartsort.transform.Localization(channel_index, torch.tensor(geom)),
            dartsort.transform.MaxAmplitude(),
        )
    )
    peeler = dartsort.peel.ThresholdAndFeaturize(
        rec,
        detection_threshold=4,
        chunk_length_samples=int(rec.sampling_frequency),
        channel_index=channel_index,
        featurization_pipeline=pipeline,
        n_chunks_fit=100,
        spatial_dedup_channel_index=torch.tensor(dartsort.make_channel_index(geom, 75.0))
    )
    peeler.load_or_fit_and_save_models(
        out_dir / "thresholding_models", n_jobs=8
    )
    thresholding_h5 = peeler.peel(
        out_dir / "thresholding.h5",
        n_jobs=8,
    )
    st = dartsort.DARTsortSorting.from_peeling_hdf5(thresholding_h5)
    me = dartsort.estimate_motion(rec, st, out_dir, amplitudes_dataset_name="ptp_amplitudes")
    dartvis.visualize_sorting(
        rec,
        st,
        out_dir / "vis",
        motion_est=me,
        make_sorting_summaries=False,
        make_animations=False,
        make_unit_summaries=False,
        amplitudes_dataset_name='ptp_amplitudes',
    )

Fit features 1.0s/it [spk/it=%%%]:   0%|          | 0/100 [00:00<?, ?it/s]

Got 73473 spikes, enough to stop early.


Threshold 4 1.0s/it [spk/it=%%%]:   0%|          | 0/600 [00:00<?, ?it/s]

Cross correlation:   0%|          | 0/3 [00:00<?, ?it/s]

/home/cwindolf/dredge/python/dredge/dredgelib.py:699: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  Nx = conv1d(onesx, weights, padding=padding)


...

Solve:   0%|          | 0/2 [00:00<?, ?it/s]

Fit features 1.0s/it [spk/it=%%%]:   0%|          | 0/100 [00:00<?, ?it/s]

Threshold 4 1.0s/it [spk/it=%%%]:   0%|          | 0/600 [00:00<?, ?it/s]

Cross correlation:   0%|          | 0/3 [00:00<?, ?it/s]

Solve:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
1

1

In [14]:
st

PosixPath('/home/cwindolf/proj/measims/dartnonn_out/Neuropixels-128_rigid_bimodal_homogeneous/thresholding.h5')

In [13]:
out_dir

PosixPath('/home/cwindolf/proj/measims/dartnonn_out/Neuropixels-128_rigid_bimodal_homogeneous')

In [ ]:
for p in darts_dir.iterdir():
    rec = si.read_binary_folder(p)
    out_dir = dart_out_dir / p.stem
    st = dartsort.dartsort(
        rec, out_dir, cfg=cfg, n_jobs_gpu=2
    )
    me = loadpkl(out_dir / "motion_est.pkl")
    dartvis.visualize_sorting(
        rec,
        st,
        out_dir / "vis",
        motion_est=me,
        make_sorting_summaries=False,
        make_animations=False,
        make_unit_summaries=False,
    )

Fit subtraction denoisers 0.9s/it [spk/it=%%%]:   0%|          | 0/100 [00:00<?, ?it/s]

/home/cwindolf/miniforge3/envs/dartsort/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
/home/cwindolf/miniforge3/envs/dartsort/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Fit features 0.9s/it [spk/it=%%%]:   0%|          | 0/100 [00:00<?, ?it/s]

/home/cwindolf/miniforge3/envs/dartsort/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
/home/cwindolf/miniforge3/envs/dartsort/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Subtraction 0.9s/it [spk/it=%%%]:   0%|          | 0/640 [00:00<?, ?it/s]

...

Subtraction 0.9s/it [spk/it=%%%]:   0%|          | 0/640 [00:00<?, ?it/s]

/home/cwindolf/miniforge3/envs/dartsort/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
/home/cwindolf/miniforge3/envs/dartsort/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Localization:   0%|          | 0/23 [00:00<?, ?it/s]

Cross correlation:   0%|          | 0/3 [00:00<?, ?it/s]

Solve:   0%|          | 0/2 [00:00<?, ?it/s]